In [4]:
import mysql.connector
import pandas as pd
import sqlparse
import sqlglot

In [23]:
database = mysql.connector.connect(
    user='root', 
    password='password',
    host='127.0.0.1', 
    port=3307,
    database="TPCH",
)

In [24]:
cursor = database.cursor()

In [25]:
statistics_queries = """
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE TABLE_SCHEMA = 'TPCH'
"""

In [26]:
keys_query = """
    SELECT 
        CASE WHEN constraint_name = "PRIMARY" THEN "PRIMARY" ELSE "FOREIGN" END AS key_type, 
        table_name, column_name, referenced_table_name, referenced_column_name
    FROM information_schema.key_column_usage 
    WHERE constraint_schema = 'TPCH' AND (constraint_name LIKE "%_ibfk_%" OR constraint_name = "PRIMARY")
"""
keys_data = pd.read_sql(keys_query, database)

/var/folders/k2/zns3zzl52ngb09pgfbbnb3pm0000gn/T/ipykernel_17158/2432916196.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  keys_data = pd.read_sql(keys_query, database)


In [41]:
table_column_info

,TABLE_NAME,COLUMN_NAME,DATA_TYPE
0,customer,c_acctbal,double
1,customer,c_address,varchar
2,customer,c_comment,varchar
3,customer,c_custkey,bigint
4,customer,c_mktsegment,char
...,...,...,...
56,supplier,s_comment,varchar
57,supplier,s_name,char
58,supplier,s_nationkey,int
59,supplier,s_phone,char


In [45]:
col_data_query = """
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE TABLE_SCHEMA = 'TPCH'
"""

table_column_info = (
    pd.read_sql(col_data_query, database)
)

schema_dictionary = dict()
for _, row in table_column_info.iterrows():
    table_name = row["TABLE_NAME"]
    if table_name not in schema_dictionary:
        schema_dictionary[table_name] = dict()
        schema_dictionary[table_name]["columns"] = dict()
    
    column_name = row["COLUMN_NAME"]

    if column_name not in schema_dictionary[table_name]:
        schema_dictionary[table_name]["columns"][column_name] = dict()
        
    schema_dictionary[table_name]["columns"][column_name]["data_type"] = str(row["DATA_TYPE"])

/var/folders/k2/zns3zzl52ngb09pgfbbnb3pm0000gn/T/ipykernel_17158/4228021697.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(col_data_query, database)


In [46]:
schema_dictionary

{'customer': {'columns': {'c_acctbal': {'data_type': 'double'},
   'c_address': {'data_type': 'varchar'},
   'c_comment': {'data_type': 'varchar'},
   'c_custkey': {'data_type': 'bigint'},
   'c_mktsegment': {'data_type': 'char'},
   'c_name': {'data_type': 'varchar'},
   'c_nationkey': {'data_type': 'int'},
   'c_phone': {'data_type': 'char'}}},
 'lineitem': {'columns': {'l_comment': {'data_type': 'varchar'},
   'l_commitdate': {'data_type': 'date'},
   'l_discount': {'data_type': 'double'},
   'l_extendedprice': {'data_type': 'double'},
   'l_linenumber': {'data_type': 'bigint'},
   'l_linestatus': {'data_type': 'char'},
   'l_orderkey': {'data_type': 'bigint'},
   'l_partkey': {'data_type': 'bigint'},
   'l_quantity': {'data_type': 'double'},
   'l_receiptdate': {'data_type': 'date'},
   'l_returnflag': {'data_type': 'char'},
   'l_shipdate': {'data_type': 'date'},
   'l_shipinstruct': {'data_type': 'char'},
   'l_shipmode': {'data_type': 'char'},
   'l_suppkey': {'data_type': 'bigi

In [47]:
keys_data

,key_type,TABLE_NAME,COLUMN_NAME,REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
0,PRIMARY,customer,c_custkey,None,None
1,FOREIGN,customer,c_nationkey,nation,n_nationkey
2,PRIMARY,lineitem,l_orderkey,None,None
3,PRIMARY,lineitem,l_linenumber,None,None
4,FOREIGN,lineitem,l_orderkey,orders,o_orderkey
5,FOREIGN,lineitem,l_partkey,partsupp,ps_partkey
6,FOREIGN,lineitem,l_suppkey,partsupp,ps_suppkey
7,PRIMARY,nation,n_nationkey,None,None
8,FOREIGN,nation,n_regionkey,region,r_regionkey
9,PRIMARY,orders,o_orderkey,None,None


In [48]:
for table_name in schema_dictionary.keys():

    count_column_string = ""
    for col in schema_dictionary[table_name]["columns"].keys():
        count_column_string += f"COUNT(DISTINCT {col}) AS distinct_{col}, SUM(ISNULL({col})) AS null_{col}, "

    count_column_string = count_column_string[:-2]
    
    counts = pd.read_sql(f"SELECT COUNT(*) AS overall_count, {count_column_string} FROM {table_name}" , database).to_dict("records")[0]

    total_records = counts["overall_count"]

    for col in counts:
        if col == "overall_count":
            schema_dictionary[table_name]["count"] = int(counts[col])
        elif col.startswith("distinct_"):
            schema_dictionary[table_name]["columns"][col.replace("distinct_", "")]["distinct_count"] = int(counts[col])
            schema_dictionary[table_name]["columns"][col.replace("distinct_", "")]["distinct_percent"] = round(100 * counts[col] / total_records, 2)

        elif col.startswith("null_"):
            schema_dictionary[table_name]["columns"][col.replace("null_", "")]["null_count"] = int(counts[col])
            schema_dictionary[table_name]["columns"][col.replace("null_", "")]["null_percent"] = round(100 * counts[col] / total_records, 2)
        else:
            raise Exception()


/var/folders/k2/zns3zzl52ngb09pgfbbnb3pm0000gn/T/ipykernel_17158/387340850.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  counts = pd.read_sql(f"SELECT COUNT(*) AS overall_count, {count_column_string} FROM {table_name}" , database).to_dict("records")[0]
/var/folders/k2/zns3zzl52ngb09pgfbbnb3pm0000gn/T/ipykernel_17158/387340850.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  counts = pd.read_sql(f"SELECT COUNT(*) AS overall_count, {count_column_string} FROM {table_name}" , database).to_dict("records")[0]
/var/folders/k2/zns3zzl52ngb09pgfbbnb3pm0000gn/T/ipykernel_17158/387340850.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or databas

In [51]:
index_count = 0
partition_count = 0
for table in schema_dictionary.keys():
    columns = schema_dictionary[table]["columns"]
    schema_dictionary[table]["index_candidates"] = list()
    schema_dictionary[table]["partition_candidates"] = list()
    for col in columns.keys():
        indexable = columns[col]["distinct_percent"] < 10
        columns[col]["indexable_flag"] = indexable
        partitionable = columns[col]["data_type"] in ["double", "int", "bigint"] and not indexable
        if indexable:
            schema_dictionary[table]["index_candidates"].append(col)
            index_count+=1
        elif partitionable:
            schema_dictionary[table]["partition_candidates"].append(col)
            partition_count+=1


    print(f"For {table}, {schema_dictionary[table]['index_candidates']} are index candidates")
    print(f"For {table}, {schema_dictionary[table]['partition_candidates']} are partition candidates")

print(f"There are {index_count} candidate index columns and {partition_count} candidate partition columns")

For customer, ['c_mktsegment', 'c_nationkey'] are index candidates
For customer, ['c_acctbal', 'c_custkey'] are partition candidates
For lineitem, ['l_commitdate', 'l_discount', 'l_linenumber', 'l_linestatus', 'l_partkey', 'l_quantity', 'l_receiptdate', 'l_returnflag', 'l_shipdate', 'l_shipinstruct', 'l_shipmode', 'l_suppkey', 'l_tax'] are index candidates
For lineitem, ['l_extendedprice', 'l_orderkey'] are partition candidates
For nation, [] are index candidates
For nation, ['n_nationkey', 'n_regionkey'] are partition candidates
For orders, ['o_clerk', 'o_custkey', 'o_orderpriority', 'o_orderstatus', 'o_shippriority'] are index candidates
For orders, ['o_orderkey', 'o_totalprice'] are partition candidates
For part, ['p_brand', 'p_container', 'p_mfgr', 'p_size', 'p_type'] are index candidates
For part, ['p_partkey', 'p_retailprice'] are partition candidates
For partsupp, ['ps_suppkey'] are index candidates
For partsupp, ['ps_availqty', 'ps_partkey', 'ps_supplycost'] are partition candi

In [57]:
for table in schema_dictionary.keys():
    for index in schema_dictionary[table]["index_candidates"]:
        index_name = f"index_{table}_{index}"
        print(f"Creating {index_name}")
        cursor.execute(f"CREATE INDEX {index_name} ON {table} ({index})")

Creating index_customer_c_mktsegment
Creating index_customer_c_nationkey
Creating index_lineitem_l_commitdate
Creating index_lineitem_l_discount
Creating index_lineitem_l_linenumber
Creating index_lineitem_l_linestatus
Creating index_lineitem_l_partkey
Creating index_lineitem_l_quantity
Creating index_lineitem_l_receiptdate
Creating index_lineitem_l_returnflag
Creating index_lineitem_l_shipdate
Creating index_lineitem_l_shipinstruct
Creating index_lineitem_l_shipmode
Creating index_lineitem_l_suppkey
Creating index_lineitem_l_tax
Creating index_orders_o_clerk
Creating index_orders_o_custkey
Creating index_orders_o_orderpriority
Creating index_orders_o_orderstatus
Creating index_orders_o_shippriority
Creating index_part_p_brand
Creating index_part_p_container
Creating index_part_p_mfgr
Creating index_part_p_size
Creating index_part_p_type
Creating index_partsupp_ps_suppkey


In [4]:
sql_reader = open("queries/test_queries.sql")
queries = sql_reader.read().split(";")
sql_reader.close()

In [7]:
queries = [sqlparse.format(query, reindent=True, keyword_case='upper') for query in queries]

In [12]:
query = queries[0]

In [69]:
%pip install sqlglot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 6.0 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
expression_tree = sqlglot.parse_one(queries[0])

In [38]:
sqlglot.parse_one("FROM lineitem USE INDEX(first_index)")

(SELECT expressions: 
  (STAR ), from: 
  (FROM this: 
    (TABLE this: 
      (IDENTIFIER this: lineitem, quoted: False), hints: 
      (INDEXTABLEHINT this: USE, expressions: 
        (IDENTIFIER this: first_index, quoted: False)))))

In [36]:
sqlglot.exp.IndexTableHint()

(INDEXTABLEHINT )

In [43]:
type(expression_tree.from_()

method

In [59]:
sqlglot.from_(expression_tree).sql()

"SELECT FROM /* using 1697822052 as a seed to the RNG */ SELECT l_returnflag, l_linestatus, SUM(l_quantity) AS sum_qty, SUM(l_extendedprice) AS sum_base_price, SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price, SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge, AVG(l_quantity) AS avg_qty, AVG(l_extendedprice) AS avg_price, AVG(l_discount) AS avg_disc, COUNT(*) AS count_order FROM lineitem WHERE l_shipdate <= CAST('1998-12-01' AS DATE) - INTERVAL '83' DAY GROUP BY l_returnflag, l_linestatus ORDER BY l_returnflag, l_linestatus"

<generator object Expression.iter_expressions at 0x11e7d9cf0>

In [88]:
list(expression_tree.walk())

[((SELECT expressions: 
    (COLUMN this: 
      (IDENTIFIER this: l_returnflag, quoted: False)), 
    (COLUMN this: 
      (IDENTIFIER this: l_linestatus, quoted: False)), 
    (ALIAS this: 
      (SUM this: 
        (COLUMN this: 
          (IDENTIFIER this: l_quantity, quoted: False))), alias: 
      (IDENTIFIER this: sum_qty, quoted: False)), 
    (ALIAS this: 
      (SUM this: 
        (COLUMN this: 
          (IDENTIFIER this: l_extendedprice, quoted: False))), alias: 
      (IDENTIFIER this: sum_base_price, quoted: False)), 
    (ALIAS this: 
      (SUM this: 
        (MUL this: 
          (COLUMN this: 
            (IDENTIFIER this: l_extendedprice, quoted: False)), expression: 
          (PAREN this: 
            (SUB this: 
              (LITERAL this: 1, is_string: False), expression: 
              (COLUMN this: 
                (IDENTIFIER this: l_discount, quoted: False)))))), alias: 
      (IDENTIFIER this: sum_disc_price, quoted: False)), 
    (ALIAS this: 
      (SUM t

In [90]:
for expression in expression_tree.walk():
    if "FROM" in str(expression):
        print(type(expression))
        print(expression)
        from_expression = expression
    if isinstance(expression, sqlglot.exp.Table):
        print('hi')
        from_expression = expression

<class 'tuple'>
((SELECT expressions: 
  (COLUMN this: 
    (IDENTIFIER this: l_returnflag, quoted: False)), 
  (COLUMN this: 
    (IDENTIFIER this: l_linestatus, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (COLUMN this: 
        (IDENTIFIER this: l_quantity, quoted: False))), alias: 
    (IDENTIFIER this: sum_qty, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (COLUMN this: 
        (IDENTIFIER this: l_extendedprice, quoted: False))), alias: 
    (IDENTIFIER this: sum_base_price, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (MUL this: 
        (COLUMN this: 
          (IDENTIFIER this: l_extendedprice, quoted: False)), expression: 
        (PAREN this: 
          (SUB this: 
            (LITERAL this: 1, is_string: False), expression: 
            (COLUMN this: 
              (IDENTIFIER this: l_discount, quoted: False)))))), alias: 
    (IDENTIFIER this: sum_disc_price, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (MUL this: 
        (MUL th

In [190]:
test = sqlglot.exp.IndexTableHint()
test.set("this", "USE")
test1 = sqlglot.exp.Identifier()
test1.set("this", "special_index")
test.set("expressions", test.expressions + [test1])
test

(INDEXTABLEHINT this: USE, expressions: 
  (IDENTIFIER this: special_index))

In [198]:
from_expression_0 = from_expression[1]

In [200]:
from_expression_0.sql()

'FROM USE INDEX (special_index)'

In [194]:
from_expression_0.set("this", test)

In [114]:
sqlglot.parse_one("FROM lineitem USE INDEX(test)")

(SELECT expressions: 
  (STAR ), from: 
  (FROM this: 
    (TABLE this: 
      (IDENTIFIER this: lineitem, quoted: False), hints: 
      (INDEXTABLEHINT this: USE, expressions: 
        (IDENTIFIER this: test, quoted: False)))))

In [195]:
from_expression_0

(TABLE this: 
  (INDEXTABLEHINT this: USE, expressions: 
    (IDENTIFIER this: special_index)))

In [110]:
test = sqlglot.expressions.From()

In [111]:
test

FROM


In [106]:
type(from_expression[1])

sqlglot.expressions.From

In [133]:
from_expression_0 = from_expression[0]

In [134]:
test = from_expression_0
print(type(test))
test

<class 'sqlglot.expressions.Table'>


(TABLE this: 
  (IDENTIFIER this: lineitem, quoted: False))

In [135]:
test = from_expression_0.this
print(type(test))
test

<class 'sqlglot.expressions.Identifier'>


(IDENTIFIER this: lineitem, quoted: False)

In [148]:
print(sqlparse.format(expression_tree.hint("USE INDEX (test)").sql(), reindent=True, keyword_case='upper') )

/* using 1697822052 as a seed to the RNG */
SELECT /*+ USE INDEX(test) */ l_returnflag,
                              l_linestatus,
                              SUM(l_quantity) AS sum_qty,
                              SUM(l_extendedprice) AS sum_base_price,
                              SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
                              SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
                              AVG(l_quantity) AS avg_qty,
                              AVG(l_extendedprice) AS avg_price,
                              AVG(l_discount) AS avg_disc,
                              COUNT(*) AS count_order
FROM lineitem
WHERE l_shipdate <= CAST('1998-12-01' AS DATE) - INTERVAL '83' DAY
GROUP BY l_returnflag,
         l_linestatus
ORDER BY l_returnflag,
         l_linestatus


"/* using 1697822052 as a seed to the RNG */ SELECT /*+ USE INDEX(test) */ l_returnflag, l_linestatus, SUM(l_quantity) AS sum_qty, SUM(l_extendedprice) AS sum_base_price, SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price, SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge, AVG(l_quantity) AS avg_qty, AVG(l_extendedprice) AS avg_price, AVG(l_discount) AS avg_disc, COUNT(*) AS count_order FROM lineitem WHERE l_shipdate <= CAST('1998-12-01' AS DATE) - INTERVAL '83' DAY GROUP BY l_returnflag, l_linestatus ORDER BY l_returnflag, l_linestatus"

In [140]:
from_expression_0.arg_types

{'this': True,
 'alias': False,
 'db': False,
 'catalog': False,
 'laterals': False,
 'joins': False,
 'pivots': False,
 'hints': False,
 'system_time': False,
 'version': False,
 'format': False,
 'pattern': False,
 'index': False}

In [ ]:
(SELECT expressions: 
  (STAR ), from: 
  (FROM this: 
    (TABLE this: 
      (IDENTIFIER this: lineitem, quoted: False), hints: 
      (INDEXTABLEHINT this: USE, expressions: 
        (IDENTIFIER this: first_index, quoted: False)))))

In [ ]:
test = from_expression_0.
print(type(test))
test

In [124]:
type(from_expression_0.this)

sqlglot.expressions.Identifier

In [209]:
expression_tree

(SELECT expressions: 
  (COLUMN this: 
    (IDENTIFIER this: l_returnflag, quoted: False)), 
  (COLUMN this: 
    (IDENTIFIER this: l_linestatus, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (COLUMN this: 
        (IDENTIFIER this: l_quantity, quoted: False))), alias: 
    (IDENTIFIER this: sum_qty, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (COLUMN this: 
        (IDENTIFIER this: l_extendedprice, quoted: False))), alias: 
    (IDENTIFIER this: sum_base_price, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (MUL this: 
        (COLUMN this: 
          (IDENTIFIER this: l_extendedprice, quoted: False)), expression: 
        (PAREN this: 
          (SUB this: 
            (LITERAL this: 1, is_string: False), expression: 
            (COLUMN this: 
              (IDENTIFIER this: l_discount, quoted: False)))))), alias: 
    (IDENTIFIER this: sum_disc_price, quoted: False)), 
  (ALIAS this: 
    (SUM this: 
      (MUL this: 
        (MUL this: 
          (C

In [281]:
table_index_info = {
    "lineitem": {
        "use_index_flag": True,
        "indexes": ["hello_world"]
    }
}

In [282]:
node = sqlglot.exp.Table()

In [283]:
node.set("this", "lineitem")

In [284]:
node.sql()

'lineitem'

In [287]:
", ".join(["a", "b", "c"])

'a, b, c'

In [288]:
table_name = node.this
if table_name in table_index_info:
    use_index_flag = table_index_info[table_name]["use_index_flag"]
    indexes = table_index_info[table_name]["indexes"]
else:
    use_index_flag = True
    indexes = list()


test = sqlglot.exp.IndexTableHint()
test.set("this", "USE")
test1 = sqlglot.exp.Identifier()
test1.set("this", ", ".join(indexes))
test.set("expressions", test.expressions + [test1])
print(test.expressions)
node.set("hints", node.expressions + [test])
print(node.sql())




[(IDENTIFIER this: hello_world)]
lineitem USE INDEX (hello_world)


In [303]:
"other" in table_index_info

False

In [310]:
huh = sqlglot.exp.Table()

In [314]:
huh.expression

In [ ]:
SELECT * FROM (SELECT COUNT(*) FROM table GROUP BY )

In [351]:
def partial_transformer(node, table_index_info):
    if isinstance(node, sqlglot.exp.Table):
        table_name = node.this.output_name
        if table_name in table_index_info:
            use_index_flag = table_index_info[table_name]["use_index_flag"]
            indexes = table_index_info[table_name]["indexes"]
        else:
            use_index_flag = True
            indexes = list()

        table_hint = sqlglot.exp.IndexTableHint()
        table_hint.set("this", "USE" if use_index_flag else "IGNORE")
        indexes_identifier = sqlglot.exp.Identifier()
        indexes_identifier.set("this", ", ".join(indexes))
        table_hint.set("expressions", table_hint.expressions + [indexes_identifier])
        node.set("hints", node.expressions + [table_hint])
        return node
    return node

In [352]:
from functools import partial

In [353]:
test_query = """
SELECT * FROM other JOIN lineitem ON "a"
"""

In [362]:
table_index_info = {
    "lineitem": {
        "use_index_flag": False,
        "indexes": ["hello_world"]
    },
    "other": {
        "use_index_flag": True,
        "indexes": ["otherwise"]
    }
}

In [2]:
from functools import partial

In [3]:
transformer = partial(partial_transformer, table_index_info=table_index_info)

NameError: name 'partial_transformer' is not defined

In [364]:
expression_tree = sqlglot.parse_one(test_query)
transformed_tree = expression_tree.transform(transformer)
print(sqlparse.format(transformed_tree.sql(), reindent=True, keyword_case='upper'))

SELECT *
FROM other USE INDEX (otherwise)
JOIN lineitem
IGNORE INDEX (hello_world) ON "a"
